In [1]:
import sqlite3
import json
from pathlib import Path
from datetime import date
import datetime as dt
import pandas as pd
from utils.global_utils import *
from st_files_connection import FilesConnection
from google.cloud import storage
from utils.my_variables import *
from collections import defaultdict
# DATA_PATH = Path('data')
# CONN = sqlite3.connect(DATA_PATH / 'account_management.db')
# CURSOR = CONN.cursor()


In [95]:
limits_df = read_csv_from_gcs('ugo_budget_limits.csv', bucket_name=BUCKET_NAME)
expenses_df, recettes_df = get_expenses_recettes('ugo')

In [30]:
limits_df.limit = 800

In [97]:
f'Total limited budget = {limits_df.limit.sum():.0f}€'

'total limited budget = 2339€'

In [31]:
date.today().replace(day=1).strftime(FRENCH_DATEFORMAT)

'2023-09-01'

In [32]:
this_month_expenses = expenses_df.sort_index()[date.today().replace(day=1).strftime(FRENCH_DATEFORMAT):].copy()

In [33]:
this_month_expenses = this_month_expenses.groupby('category').agg({'amount':'sum'})

In [34]:
limits_and_expenses = pd.merge(limits_df, this_month_expenses, right_index=True, left_on='category', how='outer').fillna(0)

In [77]:
limits_and_expenses_for_plot = limits_and_expenses.copy()

In [78]:
limits_and_expenses_for_plot['limit_reached'] = limits_and_expenses_for_plot.amount / limits_and_expenses_for_plot.limit
limits_and_expenses_for_plot['available_budget'] = limits_and_expenses_for_plot.limit - limits_and_expenses_for_plot.amount

In [84]:
limits_and_expenses_for_plot['limit_text']= limits_and_expenses_for_plot.limit.apply(lambda x: f'Limit: {x:.0f}€')
limits_and_expenses_for_plot['avail_budg_text']= limits_and_expenses_for_plot.available_budget.apply(
    lambda x: f'Left: {x:.0f}€' if x > 0 else 'Nothing left !'
    )

In [92]:
limits_df = read_csv_from_gcs('cc_budget_limits.csv', bucket_name=BUCKET_NAME)

In [93]:
len(limits_df)

0

In [91]:
import plotly.express as px
# Display the differences in a bar chart


# Define a color scale (green to red)
color_scale = ['#008000', '#FFFF00', '#FF0000']  # Green, Yellow, Red
# Create a color map based on percent values
color_map = np.interp(limits_and_expenses_for_plot.limit_reached, [0, 1], [0, 2])
bar_colors = [color_scale[int(value)] for value in color_map]



fig = go.Figure()
# Add a bar trace
proportion_bar = go.Bar(
    x=limits_and_expenses_for_plot.limit_reached,
    y=limits_and_expenses_for_plot.category,
    name='\% Limit reached',
    orientation='h',
    text=limits_and_expenses_for_plot.avail_budg_text,
    textposition='inside',
    marker=dict(color=bar_colors),
    showlegend=False
)

limit_bar = go.Bar(
    x=limits_and_expenses_for_plot.limit / limits_and_expenses_for_plot.limit,
    y=limits_and_expenses_for_plot.category,
    orientation='h',
    text=limits_and_expenses_for_plot.limit_text,
    textposition='outside',
    marker=dict(color='rgba(0,0,0,0)', line=dict(color='black', width=2)),
    showlegend=False
)

# Add both traces to the subplot
fig.add_trace(proportion_bar)
fig.add_trace(limit_bar)

# Customize the layout
fig.update_layout(
    title='Budget limits',
    xaxis=dict(title=''),  # Adjust the x-axis title
    yaxis=dict(title=''),
    barmode='overlay'  # Overlay the bars for the same category
)
fig.update_xaxes(fixedrange=True)
fig.update_yaxes(fixedrange=True)

# Show the plot
fig.show()









In [41]:
go.Bar

plotly.graph_objs._bar.Bar

In [37]:
limit_categories = defaultdict(lambda: 0)
limit_categories['a'] = 4
limit_categories['b'] = 12

In [28]:
test = pd.DataFrame(columns=['category', 'limit'])
# test['category'] = list(limit_categories.keys()) + ['c']

# for cat, lim in limit_categories.items():
#     test.limit = test.category.apply(lambda x: lim)

In [32]:
test.to_csv('ugo_budget_limits.csv', index=False)

In [33]:
pd.read_csv('ugo_budget_limits.csv')

,category,limit


In [4]:
read_csv_from_gcs('ugo_expenses.csv', bucket_name=BUCKET_NAME)

,id,date,description,category,sub_category,amount
0,1,2023-08-12,Camping séminaires Descartes,Vacances,logement,97.00
1,2,2023-08-12,Disque dur toshiba,Achats,électronique,69.99
2,3,2023-08-11,inconnu,Autres,inconnu,1.50
3,4,2023-08-08,bar descartes,Les pêchés du soir,bar,18.00
4,5,2023-08-08,tabac,Les pêchés du soir,tabac,20.10
...,...,...,...,...,...,...
182,189,2023-09-14,Part investissement Aout,Épargne,investissement,50.00
183,191,2023-09-14,Verre taff,Les pêchés du soir,bar,34.00
184,192,2023-09-14,Boissons aéroport,Vacances,alimentation,7.01
185,193,2023-09-14,Boissons aéroport,Vacances,alimentation,7.01


In [5]:
df = pd.DataFrame(columns=['id', 'date', 'description', 'category', 'sub_category', 'amount'])

In [8]:
read_csv_from_gcs('ugo_recettes.csv', bucket_name=BUCKET_NAME).columns

Index(['id', 'date', 'description', 'category', 'amount'], dtype='object')

In [1]:
import streamlit_authenticator as stauth
stauth.Hasher(['ugolegrostarpeedu17']).generate()

['$2b$12$G/Txil0/v2z9s.xBvVjxouLQVDgdkhF7iwRjDjHel8ddpEniZ.6DO']

In [1]:
import streamlit as st
from pathlib import Path
from google.cloud import storage
import json 

# Your service account key information as a dictionary
service_account_info = {
    "type": st.secrets.connections.gcs.type,
    "project_id": st.secrets.connections.gcs.project_id,
    "private_key_id": st.secrets.connections.gcs.private_key_id,
    "private_key": st.secrets.connections.gcs.private_key,
    "client_email": st.secrets.connections.gcs.client_email,
    "client_id": st.secrets.connections.gcs.client_id,
    "auth_uri": st.secrets.connections.gcs.auth_uri,
    "token_uri": st.secrets.connections.gcs.token_uri,
    "auth_provider_x509_cert_url": st.secrets.connections.gcs.auth_provider_x509_cert_url,
    "client_x509_cert_url": st.secrets.connections.gcs.client_x509_cert_url
}
# Initialize a client with your service account info
CLIENT = storage.Client.from_service_account_info(service_account_info)
DATA_PATH = Path('data')
BUCKET_NAME = "data-account-app"
USER_DEPENSES = 'ugo'
with open(DATA_PATH / 'categories.json', 'r') as json_file:
    CATEGORIES = json.load(json_file)

In [35]:
import streamlit as st
from st_files_connection import FilesConnection
from google.cloud import storage
from pathlib import Path
import sqlite3
from io import BytesIO
import io
import csv

# Your service account key information as a dictionary
service_account_info = {
    "type": st.secrets.connections.gcs.type,
    "project_id": st.secrets.connections.gcs.project_id,
    "private_key_id": st.secrets.connections.gcs.private_key_id,
    "private_key": st.secrets.connections.gcs.private_key,
    "client_email": st.secrets.connections.gcs.client_email,
    "client_id": st.secrets.connections.gcs.client_id,
    "auth_uri": st.secrets.connections.gcs.auth_uri,
    "token_uri": st.secrets.connections.gcs.token_uri,
    "auth_provider_x509_cert_url": st.secrets.connections.gcs.auth_provider_x509_cert_url,
    "client_x509_cert_url": st.secrets.connections.gcs.client_x509_cert_url
}

# Initialize a client with your service account info
CLIENT = storage.Client.from_service_account_info(service_account_info)
DATA_PATH = Path('data')
DB_NAME = 'ugo_expenses.csv'
# Replace these with your values
bucket_name = "data-account-app"

# Get the bucket
bucket = CLIENT.get_bucket(bucket_name)

# Get the blob (object) corresponding to the SQLite database file
blob = bucket.blob(DB_NAME)

# Read the CSV file directly into a DataFrame
content = blob.download_as_string()
expenses_df = pd.read_csv(BytesIO(content))

In [46]:
from utils.global_utils import *


In [40]:
new_expense = {
            'date': '1999',
            'Description': 'description',
            'Category': 'selected_catego',
            'Sub Category': 'sub_category',
            'Amount': 65
        }
        

In [43]:
expenses_df.id.max()

165.0

In [42]:
expenses_df

,id,date,description,category,sub_category,amount
0,142.0,2023-06-05,habits guerissol,Achats,vêtement,17.0
1,143.0,2023-06-05,baguettes,Alimentation,Courses,1.1
2,141.0,2023-06-05,courses,Alimentation,Courses,22.0
3,146.0,2023-06-05,bières tancrède,Les pêchés du soir,bar,13.0
4,140.0,2023-06-05,loyer Juin,Logement,Loyer,690.0
...,...,...,...,...,...,...
156,162.0,2023-08-22,courses nems etc,Alimentation,Courses,19.0
157,163.0,2023-08-22,investissements aout,Épargne,investissement,50.0
158,165.0,2023-08-22,marlusse et lapin tournée,Les pêchés du soir,bar,28.0
159,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
expenses_df.loc[len(expenses_df)] = new_expense

In [45]:
expenses_df[~expenses_df.id.isin([1,4])]

,id,date,description,category,sub_category,amount
0,142.0,2023-06-05,habits guerissol,Achats,vêtement,17.0
1,143.0,2023-06-05,baguettes,Alimentation,Courses,1.1
2,141.0,2023-06-05,courses,Alimentation,Courses,22.0
3,146.0,2023-06-05,bières tancrède,Les pêchés du soir,bar,13.0
4,140.0,2023-06-05,loyer Juin,Logement,Loyer,690.0
...,...,...,...,...,...,...
156,162.0,2023-08-22,courses nems etc,Alimentation,Courses,19.0
157,163.0,2023-08-22,investissements aout,Épargne,investissement,50.0
158,165.0,2023-08-22,marlusse et lapin tournée,Les pêchés du soir,bar,28.0
159,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.read_csv(f'gs://{bucket_name}/{DB_NAME}', encoding='utf-8')

In [9]:
# Execute a query to get all table names
CURSOR.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names using fetchall()
tables = CURSOR.fetchall()
tables

[('expenses',), ('recettes',)]

In [15]:
CURSOR.execute(f"SELECT * FROM expenses")
table_data = CURSOR.fetchall()

df1 = pd.DataFrame(table_data)

In [ ]:
new_expense = {
            'id': expenses_df.id.max() + 1,
            'date': spending_date,
            'description': description,
            'category': selected_catego,
            'sub_category': sub_category,
            'amount': amount
        }

In [11]:
df1.columns = ['id', 'date', 'description', 'category', 'sub_category', 'amount']

In [17]:
pip install streamlit-authenticator


  Preparing metadata (setup.py) ... done
  Obtaining dependency information for PyJWT>=2.3.0 from https://files.pythonhosted.org/packages/2b/4f/e04a8067c7c96c364cef7ef73906504e2f40d690811c021e1a1901473a19/PyJWT-2.8.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.4/473.4 kB 6.5 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for PyYAML>=5.3.1 from https://files.pythonhosted.org/packages/96/06/4beb652c0fe16834032e54f0956443d4cc797fe645527acee59e7deaa0a2/PyYAML-6.0.1-cp310-cp310-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for extra-streamlit-components>=0.1.60 from https://files.pythonhosted.org/packages/cc/58/96aff0e5cb8b59c06232ea7e249ed902d04ec89f52636f5be06ceb0855fe/extra_streamlit_components-0.1.60-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 11.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 6.1 MB/s eta 0:00:00
  Created wheel for streamlit-au

In [19]:
import streamlit as st
import streamlit_authenticator as stauth

In [20]:
hashed_passwords = stauth.Hasher(['Mkfarfila17!', 'def']).generate()


In [26]:

import yaml
from yaml.loader import SafeLoader
import streamlit as st
import streamlit_authenticator as stauth
with open('data/config.yaml') as file:
    config = yaml.load(file, Loader=SafeLoader)

In [27]:
config

{'credentials': {'usernames': {'jsmith': {'email': 'ugo.muhi@gmail.com',
    'name': 'Ugo Muhieddine',
    'password': '$2b$12$QzhrRLbHnmA2UsJCeMNy8eU.dpMlvmJSnpBio11yaOjLpiWlliZIe'},
   'rbriggs': {'email': 'rbriggs@gmail.com',
    'name': 'Rebecca Briggs',
    'password': '$2b$12$1z/ZbWYpKys3fFRul9nqDO3bd.BpR1XIudHB1vtCknKdwf6yG5HuK'}}},
 'cookie': {'expiry_days': 30,
  'key': 'some_signature_key',
  'name': 'some_cookie_name'},
 'preauthorized': {'emails': ['melsby@gmail.com']}}

In [ ]:
{
            'id': recettes_df.id.max() + 1,
            'date': recette_date,
            'description': description,
            'category': category,
            'amount': amount
        }

In [4]:
df1

,0,1,2,3,4,5
0,1,2023-08-12,Camping séminaires Descartes,Vacances,logement,97.00
1,2,2023-08-12,Disque dur toshiba,Achats,électronique,69.99
2,3,2023-08-11,inconnu,Autres,inconnu,1.50
3,4,2023-08-08,bar descartes,Les pêchés du soir,bar,18.00
4,5,2023-08-08,tabac,Les pêchés du soir,tabac,20.10
...,...,...,...,...,...,...
154,161,2023-08-21,kdo Antoine,Achats,cadeaux,5.20
155,162,2023-08-22,courses nems etc,Alimentation,Courses,19.00
156,163,2023-08-22,investissements aout,Épargne,investissement,50.00
157,164,2023-08-22,ticket festival rennes,Entretien personnel,culture,49.90


In [ ]:
pip install git+https://github.com/streamlit/files-connection

In [10]:
# Your service account key information as a dictionary
service_account_info = {
    "type": st.secrets.connections.gcs.type,
    "project_id": st.secrets.connections.gcs.project_id,
    "private_key_id": st.secrets.connections.gcs.private_key_id,
    "private_key": st.secrets.connections.gcs.private_key,
    "client_email": st.secrets.connections.gcs.client_email,
    "client_id": st.secrets.connections.gcs.client_id,
    "auth_uri": st.secrets.connections.gcs.auth_uri,
    "token_uri": st.secrets.connections.gcs.token_uri,
    "auth_provider_x509_cert_url": st.secrets.connections.gcs.auth_provider_x509_cert_url,
    "client_x509_cert_url": st.secrets.connections.gcs.client_x509_cert_url
}

# Initialize a client with your service account info
client = storage.Client.from_service_account_info(service_account_info)


In [12]:

# Replace these with your values
project_id = st.secrets.connections.gcs.project_id
bucket_name = "data-account-app"
object_name = "account_management_copy.db"

# Get the bucket
bucket = client.get_bucket(bucket_name)

# Get the blob (object)
blob = bucket.blob(object_name)

# Download the SQLite database file as bytes
db_file_bytes = blob.download_as_bytes()

# Open the SQLite database in memory (you can also specify a local file path)
conn = sqlite3.connect(':memory:')

In [13]:
CURSOR = CONN.cursor()

In [14]:
# Fetch all expenses from the database
CURSOR.execute(f"SELECT * FROM expenses")
table_data = CURSOR.fetchall()

df1 = pd.DataFrame(table_data)

In [15]:
df1

,0,1,2,3,4,5
0,1,2023-08-12,Camping séminaires Descartes,Vacances,logement,97.00
1,2,2023-08-12,Disque dur toshiba,Achats,électronique,69.99
2,3,2023-08-11,inconnu,Autres,inconnu,1.50
3,4,2023-08-08,bar descartes,Les pêchés du soir,bar,18.00
4,5,2023-08-08,tabac,Les pêchés du soir,tabac,20.10
...,...,...,...,...,...,...
154,161,2023-08-21,kdo Antoine,Achats,cadeaux,5.20
155,162,2023-08-22,courses nems etc,Alimentation,Courses,19.00
156,163,2023-08-22,investissements aout,Épargne,investissement,50.00
157,164,2023-08-22,ticket festival rennes,Entretien personnel,culture,49.90


In [13]:
expenses_df = get_df_from_table(CONN, CURSOR, 'expenses')
recettes_df = get_df_from_table(CONN, CURSOR, 'recettes')

In [16]:
expenses_df

,id,date,description,category,sub_category,amount
135,142,2023-06-05,habits guerissol,Achats,vêtement,17.0
136,143,2023-06-05,baguettes,Alimentation,Courses,1.1
134,141,2023-06-05,courses,Alimentation,Courses,22.0
139,146,2023-06-05,bières tancrède,Les pêchés du soir,bar,13.0
133,140,2023-06-05,loyer Juin,Logement,Loyer,690.0
...,...,...,...,...,...,...
154,161,2023-08-21,kdo Antoine,Achats,cadeaux,5.2
157,164,2023-08-22,ticket festival rennes,Entretien personnel,culture,49.9
155,162,2023-08-22,courses nems etc,Alimentation,Courses,19.0
156,163,2023-08-22,investissements aout,Épargne,investissement,50.0


In [15]:
# expenses_df.to_csv('data/ugo_expenses.csv', index=False)
# recettes_df.to_csv('data/ugo_recettes.csv', index=False)

In [19]:
# Form inputs for adding a new expense
the_id = expenses_df.id.max() + 1 
spending_date = "09-12-1999"
description = "test csv"

selected_catego = "catego_test"

sub_category = "sub catego test"
amount = 39

add_row = [the_id, spending_date, description, selected_catego, sub_category, amount]

In [23]:
expenses_df.loc[len(expenses_df)] = add_row

In [24]:
expenses_df

,id,date,description,category,sub_category,amount
135,142,2023-06-05 00:00:00,habits guerissol,Achats,vêtement,17.0
136,143,2023-06-05 00:00:00,baguettes,Alimentation,Courses,1.1
134,141,2023-06-05 00:00:00,courses,Alimentation,Courses,22.0
139,146,2023-06-05 00:00:00,bières tancrède,Les pêchés du soir,bar,13.0
133,140,2023-06-05 00:00:00,loyer Juin,Logement,Loyer,690.0
...,...,...,...,...,...,...
157,164,2023-08-22 00:00:00,ticket festival rennes,Entretien personnel,culture,49.9
155,162,2023-08-22 00:00:00,courses nems etc,Alimentation,Courses,19.0
156,163,2023-08-22 00:00:00,investissements aout,Épargne,investissement,50.0
158,165,2023-08-22 00:00:00,marlusse et lapin tournée,Les pêchés du soir,bar,28.0


In [35]:
df1.compare(df2, align_axis = 0).reset_index().level_0.unique()

array([1])

In [33]:
df1.compare(df2, align_axis=0, keep_shape=False, keep_equal=False, result_names=('self', 'other'))



1                   2       3             4      5
1 self   2023-08-12  Disque dur toshiba  Achats  électronique  69.99
  other           3                   2       3             3   4.00

In [18]:
d = {1: 'a', 2:'z'}

In [20]:
tuple([1,2])

(1, 2)

In [17]:
[col for col in ['a', 'b', 'id'] if col != 'id']

['a', 'b']

In [16]:
pd.DataFrame(table_data, columns=column_names.values())

,id,date,description,category,sub_category,amount
0,1,2023-08-12,Camping séminaires Descartes,Vacances,logement,97.00
1,2,2023-08-12,Disque dur toshiba,Achats,électronique,69.99
2,3,2023-08-11,inconnu,Autres,inconnu,1.50
3,4,2023-08-08,bar descartes,Les pêchés du soir,bar,18.00
4,5,2023-08-08,tabac,Les pêchés du soir,tabac,20.10
5,6,2023-08-07,chaussures,Achats,vêtement,10.00
6,7,2023-08-07,franprix courses,Alimentation,Courses,16.03
7,8,2023-08-07,clope electro,Achats,CE,17.80
8,9,2023-08-07,Kebab falafel,Les pêchés du soir,Kebab,17.00
9,10,2023-08-03,coupe vent rouge,Achats,vêtement,8.83


In [37]:
CONN.close()

In [26]:
with open(DATA_PATH / 'categories.json', 'r') as json_file:
    data = json.load(json_file)

In [27]:
# Create the users table if it doesn't exist
CURSOR.execute('''
    CREATE TABLE IF NOT EXISTS expenses (
    id INTEGER PRIMARY KEY,
    date DATE,
    description TEXT,
    category TEXT,
    sub_category TEXT,
    amount REAL
);
''')

In [28]:
CURSOR.execute('''
    CREATE TABLE IF NOT EXISTS recettes (
    id INTEGER PRIMARY KEY,
    date DATE,
    description TEXT,
    category TEXT,
    amount REAL
);
''')

In [31]:
# Form inputs
spending_date =  date.today()
description = "test"
category = "test"
sub_cat = "test"
amount = 74

CURSOR.execute("INSERT INTO expenses (date, description, category, sub_category, amount) VALUES (?, ?, ?, ?, ?)",
            (spending_date, description, category, sub_cat, amount))

In [33]:
# Fetch all expenses from the database
CURSOR.execute("SELECT * FROM expenses")
expenses = CURSOR.fetchall()

# Print the fetched expenses
for expense in expenses:
    print(expense)

(1, '2023-08-15', 'test', 'test', 'test', 74.0)


In [35]:
CURSOR.close(
)

In [36]:
CURSOR